In [1]:
#face detection mtcnn
#from mtcnn import MTCNN
import cv2
import numpy as np
import dlib
import imutils
from imutils import face_utils
import time
import pandas as pd
from deepface import DeepFace
from deepface.extendedmodels import Age
from deepface.commons import functions, realtime, distance as dst

#detector = MTCNN()
cap = cv2.VideoCapture(0)
cap.set(3, 1024) # set video widht
cap.set(4, 720)
# Define min window size to be recognized as a face
#minW = 0.1*cap.get(3)
#minH = 0.1*cap.get(4)
# Initializing the face detector and facial landmark predictor 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/dlib/shape_predictor_68_face_landmarks.dat")

tic = time.time()
emotion_model = DeepFace.build_model('Emotion')  
print("Emotion model loaded") 
age_model = DeepFace.build_model('Age')
print("Age model loaded")
gender_model = DeepFace.build_model('Gender')
print("Gender model loaded")
toc = time.time()
print("Facial attibute analysis models loaded in ",toc-tic," seconds")
        
while True:
    ret, frame = cap.read()
    #frame = cv2.imread("data/11.jpg")
    #frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        (x, y, w, h) = face_utils.rect_to_bb(face)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Creating an object in which we will store the detected facial landmarks
        landmarks = predictor(gray, face)     
        landmarks = face_utils.shape_to_np(landmarks)
        #for (x, y) in landmarks:
            #cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            
        #cv2.putText(frame, only_name+'/'+str("{:.2f}".format(conf)), (x, y), font, 1, (0, 0 ,255), 2,cv2.LINE_AA)

        
        gray_img = functions.preprocess_face(frame, target_size = (48, 48), grayscale = True, enforce_detection = False)
        emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
        emotion_predictions = emotion_model.predict(gray_img)[0,:]
        sum_of_predictions = emotion_predictions.sum()                      
        mood_items = []
        for i in range(0, len(emotion_labels)):
            mood_item = []
            emotion_label = emotion_labels[i]
            emotion_prediction = 100 * emotion_predictions[i] / sum_of_predictions
            mood_item.append(emotion_label)
            mood_item.append(emotion_prediction)
            mood_items.append(mood_item)                   
        emotion_df = pd.DataFrame(mood_items, columns = ["emotion", "score"])
        emotion_df = emotion_df.sort_values(by = ["score"], ascending=False).reset_index(drop=True)
        for index, instance in emotion_df.iterrows():
            emotion_label = "%s " % (instance['emotion'])
            emotion_score = instance['score']/100
        cv2.putText(frame, emotion_label, (x+w, y-h), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
                                            
        face_224 = functions.preprocess_face(frame, target_size = (224, 224),grayscale = False, enforce_detection = False)
        age_predictions = age_model.predict(face_224)[0,:]
        apparent_age = Age.findApparentAge(age_predictions)
        gender_prediction = gender_model.predict(face_224)[0,:]
        if np.argmax(gender_prediction) == 0:
            gender = "W"
        elif np.argmax(gender_prediction) == 1:
            gender = "M"
        analysis_report = str(int(apparent_age))+" "+gender
        cv2.putText(frame, analysis_report, (x+w, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 111, 255), 2)

                      
    """
    col = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pixels = np.asarray(col)  
    detections = detector.detect_faces(pixels)

    for detection in detections:
        score = detection["confidence"]
        if score > 0.90:
            x, y, w, h = detection["box"]
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    """
        
    cv2.imshow('cam',frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()

Using TensorFlow backend.


Emotion model loaded
Age model loaded
Gender model loaded
Facial attibute analysis models loaded in  6.531927824020386  seconds


In [7]:
#face detection dlib cnn

import cv2
import numpy as np
import dlib
import argparse
import time

# handle command line arguments
ap = argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True, help='path to image file')
ap.add_argument('-w', '--weights', default='data/mmod_human_face_detector.dat',
                help='path to weights file')
args = ap.parse_args()

# initialize hog + svm based face detector
hog_face_detector = dlib.get_frontal_face_detector()
# initialize cnn based face detector with the weights
cnn_face_detector = dlib.cnn_face_detection_model_v1(args.weights)


cap = cv2.VideoCapture(0)
cap.set(3, 800) # set video widht
cap.set(4, 600)
# Define min window size to be recognized as a face
#minW = 0.1*cap.get(3)
#minH = 0.1*cap.get(4)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat")

while True:
    ret, frame = cap.read()
    #frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    start = time.time()
    # apply face detection (cnn)
    faces_cnn = cnn_face_detector(gray, 1)
    end = time.time()
    print("CNN : ", format(end - start, '.2f'))
    # loop over detected faces
    for face in faces_cnn:
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
         # draw box over face
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 2)
    #cv2.putText(frame, only_name+'/'+str("{:.2f}".format(conf)), (x, y), font, 1, (0, 0 ,255), 2,cv2.LINE_AA)
        
    cv2.imshow('cam',frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()

usage: ipykernel_launcher.py [-h] -i IMAGE [-w WEIGHTS]
ipykernel_launcher.py: error: the following arguments are required: -i/--image


SystemExit: 2

In [ ]:
# import required packages
import cv2
import dlib
import argparse
import time
import easydict
import sys
from sys import argv



# handle command line arguments
ap = argparse.ArgumentParser()
ap.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
#ap.add_argument('-i', '--image', required=True, help='path to image file')
ap.add_argument('-w', '--weights', default='data/dlib/mmod_human_face_detector.dat', help='path to weights file')
#args = ap.parse_args()

args = ap.parse_args(sys.argv[1:])


cnn_face_detector = dlib.cnn_face_detection_model_v1(args.weights)

# load input image
cap = cv2.VideoCapture(0)
while True :
    ret, frame = cap.read()
    #image = cv2.imread(args.image)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    start = time.time()
    # apply face detection (cnn)
    faces_cnn = cnn_face_detector(image)
    end = time.time()
    print("CNN : ", format(end - start, '.2f'))

    # loop over detected faces
    for face in faces_cnn:
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
        # draw box over face
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 2)
        #cv2.putText(image, "CNN", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,255),1,cv2.LINE_AA)
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cv2.imshow("cnn_face_detection", frame)
# close all windows
cap.release()
cv2.destroyAllWindows()